<a href="https://colab.research.google.com/github/chang-ho-jeon/stock_data_anaysis/blob/main/RL_LSTM%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%EC%A3%BC%EA%B0%80%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gym
from sklearn.preprocessing import MinMaxScaler

# 파라미터 설정
symbol = "005930.KS"  # 삼성전자 티커
start_date = "2024-01-01"
end_date = "2024-10-29"

# 데이터 다운로드
data = yf.download(symbol, start=start_date, end=end_date)
data = data[['Close']]  # 종가 데이터만 사용

# 데이터 전처리
scaler = MinMaxScaler(feature_range=(-1, 1))
data['Close'] = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# 학습 데이터 준비
window_size = 60  # LSTM 입력으로 사용할 시퀀스 길이
train_data = []
target_data = []

for i in range(len(data) - window_size):
    train_data.append(data['Close'].values[i:i + window_size])
    target_data.append(data['Close'].values[i + window_size])

train_data = torch.FloatTensor(train_data)
target_data = torch.FloatTensor(target_data)

# LSTM 모델 정의
class RL_LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=50, output_size=1):
        super(RL_LSTM, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

# 모델 초기화
model = RL_LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
epochs = 100
for i in range(epochs):
    for seq, labels in zip(train_data, target_data):
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

# 예측 수행
model.eval()
with torch.no_grad():
    test_inputs = data['Close'].values[-window_size:].tolist()
    for _ in range(1):  # 다음 날 예측
        seq = torch.FloatTensor(test_inputs[-window_size:])
        with torch.no_grad():
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                                 torch.zeros(1, 1, model.hidden_layer_size))
            test_inputs.append(model(seq).item())

# 예측 값 변환
predicted_price = scaler.inverse_transform(np.array(test_inputs[-1]).reshape(-1, 1))
print(f'예측된 내일 종가: {predicted_price[0][0]:.2f} 원')


[*********************100%***********************]  1 of 1 completed
<ipython-input-1-0674a8c2246c>:32: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  train_data = torch.FloatTensor(train_data)


epoch:   0 loss: 0.00679507
epoch:  10 loss: 0.00587226
epoch:  20 loss: 0.00859243
epoch:  30 loss: 0.01325341
epoch:  40 loss: 0.01108073
epoch:  50 loss: 0.00943975
epoch:  60 loss: 0.00882545
epoch:  70 loss: 0.01364411
epoch:  80 loss: 0.00373788
epoch:  90 loss: 0.00453738
예측된 내일 종가: 57486.31 원
